In [1]:
#import libraries
import json
from datetime import datetime, date
import requests
import urllib.parse
import urllib.request
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse

In [10]:
#example request from dialogflow cx
request_json = {
  "queryResult": {
    "queryText": "What is the weather in Amsterdam on Friday?",
    "parameters": {
      "date_day": { "day": 4, "month": 3, "year": 2023 }, 
      "location_city": "Amsterdam"
    }
  }
}

In [16]:
def weather(request_json):
    city_name = request_json['queryResult']['parameters']['location_city']

    query_date = request_json['queryResult']['parameters']['date_day']
    query_date = date(query_date['year'], query_date['month'], query_date['day'])

    api_key = "****************"
    endpoint = 'https://api.openweathermap.org/data/2.5/forecast'

    query_params = {
    'q': city_name,
    'units': 'metric',
    'appid': api_key
    }

    #send API request and retrieve JSON response with 'requests'
    # response = requests.get(endpoint, params=query_params)
    # weather_data = response.json()

    #encode query parameters with urllib
    query_string = urllib.parse.urlencode(query_params)
    #send API request and retrieve JSON response
    response = urllib.request.urlopen(endpoint + '?' + query_string)
    weather_data = json.loads(response.read().decode())
    
    #create list with dicts with all forecast data
    forecast_data = []

    for forecast in weather_data['list']:
        forecast_date = datetime.fromtimestamp(forecast['dt'])
        forecast_time = forecast['dt_txt'].split()[1]
        forecast_temp_min = forecast['main']['temp_min']
        forecast_temp_max = forecast['main']['temp_max']
        forecast_data.append({'date': forecast_date, 'time': forecast_time, 'temp_min': forecast_temp_min, 'temp_max': forecast_temp_max})

    #collect all the minimum and maximum temperatures of day
    min_temps = []
    max_temps = []

    for datestamp in forecast_data:
        #check data for the query date
        if datestamp['date'].date() == query_date:
            min_temps.append(datestamp['temp_min'])
            max_temps.append(datestamp['temp_max'])

    #calculate the minimum and maximum temperature of day
    min_temp = min(min_temps)
    max_temp = max(max_temps)
    
    #create response text
    response_text = "On " + str(query_date) + " in " + city_name + " the minimum temperature is " + str(min_temp) + "°C and the maximum temperature is " + str(max_temp) + "°C"
    
    #dump retrieved data in appropiate json format
    response = json.dumps(
            {
                "fulfillment_response": {
                    "messages": [
                        {
                            "text": {
                                "text": [f"Minimum temperature: {min_temp} Maximum temperature: {max_temp}"],
                            }
                        }
                    ]
                }
            }
        )
    
    return(response)

In [17]:
weather(request_json)

'{"fulfillment_response": {"messages": [{"text": {"text": ["Minimum temperature: 3.78 Maximum temperature: 6.88"]}}]}}'